# Create COVID CASES list

Create a list of covid cases for T days, chosing a T and an online repository of covid cases. 

In this case, we are looking for a relatively small population dataset, so we choose the smallest canton of swizterland (12'800 people). 

In [1]:
# Basic math libraries
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import math
from matplotlib import pyplot as plt
import pickle
from matplotlib.colors import ListedColormap

# Matrix eigendecomposition
import numpy.linalg as lg
import scipy.linalg as slg

# Networks and Graphs
import networkx as nx
import dynetx as dn
import graphviz 
from networkx.drawing.nx_agraph import graphviz_layout

# Random data generation
import itertools
import random
from collections import Counter
from random import randint
from itertools import chain

# Neural Networks
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

%load_ext autoreload
%autoreload 2
import time

In [2]:
# take data from github

github_url = 'https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_cases_switzerland_openzh.csv'     
file = pd.read_csv(github_url+"?raw=true")
file.head(10)

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,...,VS_diff_pc,ZG_diff,ZG_pc,ZG_diff_pc,ZH_diff,ZH_pc,ZH_diff_pc,CH_diff,CH_pc,CH_diff_pc
0,2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.024601e-07,NaN
1,2020-02-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.287844e-06,5.853834e-07
2,2020-02-27,1.0,NaN,NaN,NaN,NaN,1.0,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000001,NaN,5.0,1.873227e-06,5.853834e-07
3,2020-02-28,1.0,NaN,NaN,1.0,1.0,1.0,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000001,0.000000e+00,10.0,3.043994e-06,1.170767e-06
4,2020-02-29,3.0,NaN,NaN,NaN,2.0,1.0,0.0,14.0,NaN,...,0.000000,NaN,0.000000,NaN,4.0,0.000004,2.629904e-06,13.0,4.565991e-06,1.521997e-06
5,2020-03-01,4.0,NaN,NaN,2.0,2.0,1.0,1.0,15.0,NaN,...,0.000003,0.0,0.000000,0.000000,1.0,0.000005,6.574760e-07,8.0,5.502604e-06,9.366135e-07
6,2020-03-02,7.0,NaN,NaN,4.0,2.0,1.0,2.0,16.0,NaN,...,0.000006,0.0,0.000000,0.000000,3.0,0.000007,1.972428e-06,19.0,7.727061e-06,2.224457e-06
7,2020-03-03,7.0,NaN,NaN,NaN,2.0,3.0,2.0,19.0,NaN,...,0.000000,1.0,0.000008,0.000008,3.0,0.000009,1.972428e-06,14.0,9.366135e-06,1.639074e-06
8,2020-03-04,10.0,NaN,NaN,6.0,2.0,3.0,4.0,21.0,NaN,...,0.000003,0.0,0.000008,0.000000,2.0,0.000010,1.314952e-06,30.0,1.287844e-05,3.512301e-06
9,2020-03-05,11.0,NaN,1.0,NaN,6.0,8.0,6.0,26.0,NaN,...,0.000003,1.0,0.000016,0.000008,8.0,0.000015,5.259808e-06,46.0,1.826396e-05,5.385528e-06


In [3]:
# clean the file

AI = file[["Date", "AI"]]
covid_cases = AI.rename(columns={"AI": "cases"})
covid_cases = covid_cases.fillna(method='ffill')              # fill down the number of covid cases in the dates where NaN
covid_cases = covid_cases.dropna().reset_index(drop=True)     # remove the first NaN values and reset the index
covid_cases = covid_cases.astype({'cases': 'int32'})          # cast as integer the column with number of cases

In [4]:
# create a list and save it as npy file into input folder

covid_cases_list = list(covid_cases["cases"])                 # convert to list
np.save("datasets/inputs/AI_covid_cases.npy", covid_cases_list)

In [5]:
# PIECE TO ADD IN THE 2. FILE

# import covid cases and take the first T days

T = 365
covid_cases = list(np.load("datasets/inputs/AI_covid_cases.npy", allow_pickle=True))[:T]
covid_cases

[2,
 2,
 4,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 8,
 9,
 11,
 12,
 13,
 13,
 14,
 14,
 14,
 20,
 20,
 21,
 21,
 21,
 21,
 23,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 27,
 27,
 28,
 31,
 31,
 31,
 31,
 33,
 33,
 36,
 36,
 36,
 36,
 36,
 39,
 39,
 40,
 41,
 4